## Sample Syntax



In [ ]:
using SpikeSorting

### Creating Sorting Structure

The core of SpikeSorting.jl is the Sorting data structure. It is defined by how the algorithm will perform
* Detection
* Clustering
* Alignment
* Feature extraction
* Dimensionality Reduction
* Thresholding

As well as the number of channels

In [ ]:
detect=DetectPower();
cluster=ClusterOSort();
align=AlignMax();
feature=FeatureTime();
reduce=ReductionNone();
thres=ThresholdMean();
num_channels=20;

s1=create_multi(detect,cluster,align,feature,reduce,thres,num_channels);

### Creating Output Buffers

The data structures that contain the detected spikes and input voltages are separate from the sorting structure to facilitate integration with other software. The output_buffer method will create the data structure that hold:

* Detected Spikes
* Number of detected spikes on each channel

The idea is that these buffers will be wiped after each iteration of online sorting.

In [ ]:
(buf,nums)=output_buffer(num_channels);

### Calibration

Before sorting, there are several types of calibration that need to occur. First, the very first iteration will initialize all methods that need some kind of starting values. Then, the thresholds for detection need to be calculated. Then any candidate spikes that are detected may need to be used to calibrate dimensionality reduction or clustering steps. 

The cal! method will perform the appropriate stage of calibration based on the value of its last input as follows:

* 0 = first run
* 1 = threshold calibration
* 2 = dimensionality reduction / clustering calibration

So for online use, a timer or iteration counter could be used so that time is spent in each of the appropriate calibration phases.

In [ ]:
v=rand(1:1000, 1000, num_channels); #Dummy voltage matrix

#First run
cal!(s1,v,buf,nums,0)

#Threshold calibration
cal!(s1,v,buf,nums,1)

#Clustering / DM calibration
cal!(s1,v,buf,nums,2)

### Online Sorting

Online Sorting is performed in a similar fashion, with the onlinesort! method:

In [ ]:
onlinesort!(s1,v,buf,nums);